# Bond Pricing

This notebook replicates the QuantLib `Bonds` example: build a yield curve from bond and deposit/swap market data, then price zero-coupon, fixed-rate, and floating-rate bonds.

In [1]:
import pyquantlib as ql
from datetime import date

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

PyQuantLib 0.1.0 (QuantLib 1.40)


## Setup

In [2]:
calendar = ql.TARGET()
settlement_date = date(2008, 9, 18)
settlement_days = 3

todays_date = calendar.advance(settlement_date, -settlement_days, ql.Days)
ql.Settings.evaluationDate = todays_date

print(f"Today:      {todays_date}")
print(f"Settlement: {settlement_date}")

Today:      September 15th, 2008
Settlement: 2008-09-18


## Building the Bond Discounting Curve

Bootstrap a yield curve from five Treasury bonds using `FixedRateBondHelper`.

In [3]:
redemption = 100.0

bond_data = [
    # (issue_date, maturity, coupon, market_price)
    (date(2005, 3, 15), date(2010, 8, 31), 0.02375, 100.390625),
    (date(2005, 6, 15), date(2011, 8, 31), 0.04625, 106.21875),
    (date(2006, 6, 30), date(2013, 8, 31), 0.03125, 100.59375),
    (date(2002, 11, 15), date(2018, 8, 15), 0.04000, 101.6875),
    (date(1987, 5, 15), date(2038, 5, 15), 0.04500, 102.140625),
]

bond_helpers = []
for issue, maturity, coupon, price in bond_data:
    schedule = ql.Schedule(
        issue, maturity, ql.Period(ql.Semiannual), calendar,
        ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False,
    )
    helper = ql.FixedRateBondHelper(
        price, settlement_days, 100.0, schedule, [coupon],
        ql.ActualActual(ql.ActualActual.Bond),
        ql.Unadjusted, redemption, issue,
    )
    bond_helpers.append(helper)

ts_day_counter = ql.Actual365Fixed()
bond_curve = ql.PiecewiseLogLinearDiscount(todays_date, bond_helpers, ts_day_counter)

print(f"Bond curve built with {len(bond_helpers)} helpers")
for issue, maturity, coupon, price in bond_data:
    print(f"  {coupon:.3%} {maturity}  @ {price:.4f}")

Bond curve built with 5 helpers
  2.375% 2010-08-31  @ 100.3906
  4.625% 2011-08-31  @ 106.2188
  3.125% 2013-08-31  @ 100.5938
  4.000% 2018-08-15  @ 101.6875
  4.500% 2038-05-15  @ 102.1406


## Building the Euribor Forecasting Curve

Bootstrap from a 6M deposit and swap rates (2Y, 3Y, 5Y, 10Y, 15Y).

In [4]:
# 6M deposit
d6m = ql.DepositRateHelper(0.03385, ql.Period("6M"), 2, calendar,
                            ql.ModifiedFollowing, True, ql.Actual360())

# Swap conventions
sw_fixed_freq = ql.Annual
sw_fixed_conv = ql.Unadjusted
sw_fixed_dc = ql.Thirty360(ql.Thirty360.European)
sw_float_index = ql.Euribor6M()

swap_data = [
    ("2Y", 0.0295),
    ("3Y", 0.0323),
    ("5Y", 0.0359),
    ("10Y", 0.0412),
    ("15Y", 0.0433),
]

depo_swap_helpers = [d6m]
for tenor, rate in swap_data:
    depo_swap_helpers.append(
        ql.SwapRateHelper(rate, ql.Period(tenor), calendar,
                          sw_fixed_freq, sw_fixed_conv, sw_fixed_dc,
                          sw_float_index)
    )

spot_date = calendar.advance(todays_date, 2, ql.Days)
forecasting_curve = ql.PiecewiseLogLinearDiscount(
    spot_date, depo_swap_helpers, ts_day_counter)

print(f"Forecasting curve built from deposit + {len(swap_data)} swaps")

Forecasting curve built from deposit + 5 swaps


## Bond Pricing

Price three bonds: zero-coupon, fixed-rate, and floating-rate.

In [5]:
# Relinkable handles for discounting and forecasting
discounting_handle = ql.RelinkableYieldTermStructureHandle(bond_curve)
forecasting_handle = ql.RelinkableYieldTermStructureHandle(forecasting_curve)

face_amount = 100.0
bond_engine = ql.DiscountingBondEngine(discounting_handle)

In [6]:
# Zero coupon bond
zero_coupon_bond = ql.ZeroCouponBond(
    settlement_days, calendar, face_amount,
    date(2013, 8, 15), ql.Following, 116.92,
    date(2003, 8, 15),
)
zero_coupon_bond.setPricingEngine(bond_engine)

# Fixed rate bond (4.5% annual)
fixed_schedule = ql.Schedule(
    date(2007, 5, 15), date(2017, 5, 15),
    ql.Period(ql.Annual), calendar,
    ql.Unadjusted, ql.Unadjusted,
    ql.DateGeneration.Backward, False,
)
fixed_rate_bond = ql.FixedRateBond(
    settlement_days, face_amount, fixed_schedule, [0.045],
    ql.ActualActual(ql.ActualActual.Bond),
    ql.ModifiedFollowing, 100.0, date(2007, 5, 15),
)
fixed_rate_bond.setPricingEngine(bond_engine)

# Floating rate bond (Euribor 6M + 10bp)
euribor6m = ql.Euribor6M(forecasting_handle)
euribor6m.addFixing(date(2007, 10, 18), 0.026)
euribor6m.addFixing(date(2008, 4, 17), 0.028)

float_schedule = ql.Schedule(
    date(2005, 10, 21), date(2010, 10, 21),
    ql.Period(ql.Semiannual), calendar,
    ql.Unadjusted, ql.Unadjusted,
    ql.DateGeneration.Backward, True,
)
floating_rate_bond = ql.FloatingRateBond(
    settlement_days, face_amount, float_schedule, euribor6m,
    ql.Actual360(), ql.ModifiedFollowing, 2,
    [1.0], [0.001],  # gearings, spreads
)
floating_rate_bond.setPricingEngine(bond_engine)

pricer = ql.BlackIborCouponPricer()
ql.setCouponPricer(floating_rate_bond.cashflows(), pricer)

print("Bonds created: ZeroCoupon, FixedRate (4.5%), FloatingRate (Euribor6M + 10bp)")

Bonds created: ZeroCoupon, FixedRate (4.5%), FloatingRate (Euribor6M + 10bp)


## Results

In [7]:
bonds = [
    ("ZC", zero_coupon_bond),
    ("Fixed", fixed_rate_bond),
    ("Floating", floating_rate_bond),
]

print(f"{'':18s} {'ZC':>10s} {'Fixed':>10s} {'Floating':>10s}")
print("-" * 48)

row = lambda label, fn: print(
    f"{label:18s} {fn(zero_coupon_bond):10.2f} {fn(fixed_rate_bond):10.2f} {fn(floating_rate_bond):10.2f}"
)

row("Net present value", lambda b: b.NPV())
row("Clean price", lambda b: b.cleanPrice())
row("Dirty price", lambda b: b.dirtyPrice())
row("Accrued coupon", lambda b: b.accruedAmount())

print(f"{'Previous coupon':18s} {'N/A':>10s} {fixed_rate_bond.previousCouponRate():10.2%} {floating_rate_bond.previousCouponRate():10.2%}")
print(f"{'Next coupon':18s} {'N/A':>10s} {fixed_rate_bond.nextCouponRate():10.2%} {floating_rate_bond.nextCouponRate():10.2%}")

row("Yield", lambda b: b.bondYield(ql.Actual360(), ql.Compounded, ql.Annual))

                           ZC      Fixed   Floating
------------------------------------------------
Net present value      100.92     107.32     102.85
Clean price            100.93     105.79     101.66
Dirty price            100.93     107.34     102.87
Accrued coupon           0.00       1.55       1.21
Previous coupon           N/A      4.50%      2.70%
Next coupon               N/A      4.50%      2.90%
Yield                    0.03       0.04       0.02


## Yield / Price Conversions

In [8]:
frn_yield = floating_rate_bond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual)
frn_clean = floating_rate_bond.cleanPrice()

price_from_yield = floating_rate_bond.cleanPrice(
    frn_yield, ql.Actual360(), ql.Compounded, ql.Annual, settlement_date)

yield_from_price = floating_rate_bond.bondYield(
    ql.BondPrice(frn_clean, ql.BondPriceType.Clean),
    ql.Actual360(), ql.Compounded, ql.Annual, settlement_date)

print(f"Floating rate bond:")
print(f"  Yield to clean price: {price_from_yield:.6f}")
print(f"  Clean price to yield: {yield_from_price:.6%}")

Floating rate bond:
  Yield to clean price: 101.662905
  Clean price to yield: 2.202173%
